In [1]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM
from nltk.tokenize import sent_tokenize
import nltk
import time
import pickle
from torch.utils.data import DataLoader
from torch.optim import AdamW
import random

path = "/global/cscratch1/sd/ajaybati/model.pt"


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
modelasdf = BertForMaskedLM.from_pretrained('bert-base-uncased')
# model.load_state_dict(torch.load(path))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def bleu(p,r):
    smoothie = SmoothingFunction().method2
    bleu_list = []
    for index in range(len(p)):
        BLEUscore = nltk.translate.bleu_score.sentence_bleu(p[index],r[index],smoothing_function=smoothie)
        bleu_list.append(BLEUscore)
    return sum(bleu_list) / len(bleu_list)
def getSent_pred(prediction,real_labels): #convert all ids to sentences for bscore
    sentlist_real = []
    sep_list = []
    for sent2 in real_labels:
        tokenized = tokenizer.convert_ids_to_tokens(sent2)
        sep = tokenized.index('[SEP]')
        sep_list.append(sep)
        sentlist_real.append(tokenized[1:sep])
    
    
    sentlist_ids = []
    sentlist = []
    for sent in prediction:
        word_list = []
        for word in sent:
            word_list.append(torch.argmax(word))
        sentlist_ids.append(word_list)
    
    for index,sent in enumerate(sentlist_ids):
        sentlist.append(tokenizer.convert_ids_to_tokens(sent)[1:sep_list[index]])
    return sentlist,sentlist_real
def calc_accuracy(prediction, real_labels, mask_indices):
    score = 0
    total = 0
    for step,sent in enumerate(mask_indices):
        if list(sent).count(0)!=40:
            for mask in sent:
                if int(mask)!=0:
                    predicted_index = int(torch.argmax(prediction[step,int(mask)]))
                    actual = int(real_labels[step][int(mask)])
                    if bool(predicted_index==actual):
                        score+=1
                    total+=1
                else:
                    pass

        else:
            pass
    
    p,r = getSent_pred(prediction,real_labels)
    
    
    accuracy = score/total
    try:
        bscore = bleu(p,r)
    except:
        bscore = "Unfortunately, not possible"
    return accuracy, bscore 


In [16]:
def bert_output(sents,n_percent_mask):
    input_ids_real = []
    att = []
    compare = []
    mask_indices = []
    for sent in sent_tokenize(sents):
        mask = []
        encoded_dict = tokenizer.encode_plus(
            sent,                      # Sentence to encode.
            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
            max_length = 64,
            truncation = True,# Pad & truncate all sentences.
            pad_to_max_length = True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',     # Return pytorch tensors.
       )

        # Add the encoded sentence to the list.    
        input_ids = encoded_dict['input_ids']
        compare.append(input_ids)
        attention_masks = encoded_dict['attention_mask']
        att.append(attention_masks)
        input_ids_part = []
        for step,word in enumerate(input_ids[0]):
            if int(word) != 101 and int(word) != 102:
                rando = random.random()
                random.seed()
                if rando < n_percent_mask and int(word)!=0:
                    mask.append(step)
                    input_ids_part.append(103)
                else:
                    input_ids_part.append(int(word))
            else:
                input_ids_part.append(int(word))
        input_ids_part = torch.tensor(input_ids_part).view(1,64)
        input_ids_real.append(input_ids_part)
        mask_indices.append(mask)
    
    if len(input_ids_real)>1:
        bert_input = torch.cat(tuple(input_ids_real),0)
        att = torch.cat(tuple(att),0)
        compare = torch.cat(tuple(compare),0)
    else:
        bert_input = input_ids_real[0]
        att = att[0]
        compare = compare[0]
        
    
    loss, predictions = modelasdf(bert_input,attention_mask = att, masked_lm_labels = compare)
    
    accuracy, bscore = calc_accuracy(predictions, compare, mask_indices)
    
    
    return loss,predictions, accuracy, bscore
        


yo = bert_output("hi, i love to eat chicken during thanksgiving feasts.",0.2)
yo

(tensor(15.2788, grad_fn=<NllLossBackward>),
 tensor([[[ -6.7765,  -6.7445,  -6.7448,  ...,  -6.1193,  -5.9265,  -4.0404],
          [ -5.9526,  -5.8347,  -5.8903,  ...,  -6.0357,  -6.4135,  -4.6279],
          [-13.9339, -13.6718, -13.6458,  ..., -12.6302, -11.9930, -12.1524],
          ...,
          [ -8.1411,  -8.0170,  -8.2725,  ...,  -7.6747,  -7.6799,  -7.8712],
          [ -6.8351,  -6.8551,  -6.8548,  ...,  -7.7234,  -6.6308,  -4.8302],
          [ -7.9110,  -7.9581,  -7.6260,  ...,  -8.2757,  -6.6852,  -6.5242]]],
        grad_fn=<AddBackward0>),
 0.3333333333333333,
 'Unfortunately, not possible')